## Classifying CIFAR-10 with Data Augmentation

In this exercise, we revisit CIFAR-10 and the networks we previously built.  We will use real-time data augmentation to try to improve our results.

When you are done going through the notebook, experiment with different data augmentation parameters and see if they help (or hurt!) the performance of your classifier.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In Exercise 6, we built two models.  One was smaller (181K parameters) while the second was larger (1.25M) parameters.  Below we use the smaller model and train it with data augmentation.

In [5]:
# Let's build a CNN using Keras' Sequential capabilities

model_1 = Sequential()


## 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

## Another 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

## 2x2 max pooling reduces to 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

## Flatten turns 3x3x32 into 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        2432      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          25632     
_________________________________________________________________
activation_2 (Activation)    (None, 6, 6, 32)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)               0         
__________

We still have 181K parameters, even though this is a "small" model.


In [6]:
batch_size = 32

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0005, decay=1e-6)

# Let's train the model using RMSprop
model_1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Here we define the `ImageDataGenerator` that we will use to serve images to our model during the training process.  Currently, it is configured to do some shifting and horizontal flipping.

In [7]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
                   
datagen.fit(x_train)      # This computes any statistics that may be needed (e.g. for centering) from the training set.

    
# Fit the model on the batches generated by datagen.flow().
model_1.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=15,
                    validation_data=(x_test, y_test))

Epoch 1/15
1562/1562 [==============================] - 25s 16ms/step - loss: 1.8023 - acc: 0.3349 - val_loss: 1.4989 - val_acc: 0.4571
Epoch 2/15
1562/1562 [==============================] - 23s 15ms/step - loss: 1.5507 - acc: 0.4393 - val_loss: 1.3508 - val_acc: 0.5177
Epoch 3/15
1562/1562 [==============================] - 23s 15ms/step - loss: 1.4633 - acc: 0.4749 - val_loss: 1.2547 - val_acc: 0.5567
Epoch 4/15
1562/1562 [==============================] - 23s 15ms/step - loss: 1.4016 - acc: 0.4959 - val_loss: 1.3667 - val_acc: 0.5272
Epoch 5/15
1562/1562 [==============================] - 23s 15ms/step - loss: 1.3646 - acc: 0.5140 - val_loss: 1.3051 - val_acc: 0.5488
Epoch 6/15
1562/1562 [==============================] - 23s 15ms/step - loss: 1.3397 - acc: 0.5233 - val_loss: 1.2222 - val_acc: 0.5661
Epoch 7/15
1562/1562 [==============================] - 23s 15ms/step - loss: 1.3250 - acc: 0.5286 - val_loss: 1.3448 - val_acc: 0.5394
Epoch 8/15
1562/1562 [==========================

How does the performance compare with the non-augmented training?

    Non-augumented training: loss: 1.0782 - acc: 0.6312 - val_loss: 1.1706 - val_acc: 0.6049

    Augumentend training: loss: 1.2790 - acc: 0.5547 - val_loss: 1.1996 - val_acc: 0.5841

## Exercise
### Your Turn

1. Experiment above with different settings of the data augmentation parameters.  Can you make the model do better?  Can you make it do worse?

2. As in Exercise 6, Build a more complicated model with the following pattern:
    - Conv -> Conv -> MaxPool -> Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Final Classification
    - Use strides of 1 for all convolutional layers.

3. Use data augmentation to train this model.  Can you get better performance?

In [8]:
# Let's build a CNN using Keras' Sequential capabilities

# Please provide your code here

model_2 = Sequential()
model_2.add(Conv2D(32, (3, 3), strides = (1,1), padding='same',
                 input_shape=x_train.shape[1:]))
model_2.add(Activation('relu'))
model_2.add(Conv2D(32, (3, 3), strides = (1,1)))
model_2.add(Activation('relu'))

model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.25))

model_2.add(Conv2D(64, (5, 5), strides = (1,1), padding='same',
                 input_shape=x_train.shape[1:]))
model_2.add(Activation('relu'))
model_2.add(Conv2D(64, (5, 5), strides = (1,1)))
model_2.add(Activation('relu'))

model_2.add(Flatten())
model_2.add(Dense(512))
model_2.add(Activation('relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(num_classes))
model_2.add(Activation('softmax'))

## Check number of parameters (print the summary)
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 15, 15, 64)        51264     
__________

In [9]:
# initiate RMSprop optimizer
batch_size = 32
opt = keras.optimizers.rmsprop(lr=0.0005, decay=1e-6)

# Let's train the model using RMSprop
model_2.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [10]:
# Compute quantities required for feature-wise normalization
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
                   
datagen.fit(x_train)      # This computes any statistics that may be needed (e.g. for centering) from the training set.

    
# Fit the model on the batches generated by datagen.flow().
model_2.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=15,
                    validation_data=(x_test, y_test))

Epoch 1/15
1562/1562 [==============================] - 24s 15ms/step - loss: 1.5614 - acc: 0.4339 - val_loss: 1.1219 - val_acc: 0.5955
Epoch 2/15
1562/1562 [==============================] - 23s 15ms/step - loss: 1.1559 - acc: 0.5912 - val_loss: 1.1410 - val_acc: 0.6111
Epoch 3/15
1562/1562 [==============================] - 23s 15ms/step - loss: 1.0158 - acc: 0.6493 - val_loss: 0.9478 - val_acc: 0.6826
Epoch 4/15
1562/1562 [==============================] - 23s 15ms/step - loss: 0.9551 - acc: 0.6757 - val_loss: 0.8431 - val_acc: 0.7193
Epoch 5/15
1562/1562 [==============================] - 24s 15ms/step - loss: 0.9250 - acc: 0.6890 - val_loss: 0.8781 - val_acc: 0.7142
Epoch 6/15
1562/1562 [==============================] - 23s 15ms/step - loss: 0.9208 - acc: 0.6957 - val_loss: 0.8939 - val_acc: 0.7141
Epoch 7/15
1562/1562 [==============================] - 23s 15ms/step - loss: 0.9261 - acc: 0.6995 - val_loss: 1.3062 - val_acc: 0.6762
Epoch 8/15
1562/1562 [==========================

### Answers   

    1. Yes, I can make the model do better.

    3. When I used data augumentation to this model, I get better results.
    
    Non-augumented training: loss: 1.0782 - acc: 0.6312 - val_loss: 1.1706 - val_acc: 0.6049

    Augumentend training: loss: 1.2790 - acc: 0.5547 - val_loss: 1.1996 - val_acc: 0.5841
    
    My model with augumentation: loss: 1.1300 - acc: 0.6467 - val_loss: 0.9824 - val_acc: 0.6840